# Подготовка текстов

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Библиотека `transformers` (<a href='https://huggingface.co/transformers/'>huggingface</a>) позволяет разбить тексты выборки на специальные токены, которые используются в модели BERT. Рассмотрим мини-батч, в который попали два текста: "I love Pixar.", "I don't care for Pixar.", принадлежающие к классам 1 и 0 соответственно. Разобьем имеющиеся тексты на токены:

In [ ]:
from transformers import BertTokenizer
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text_batch = ["I love Pixar.", "I don't care for Pixar."]
labels = torch.tensor([1,0]).unsqueeze(0)

encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

Для обучения модели нам понадобится объект `'input_ids'`, содержащий id каждого из токенов, входящих в текст, а также объект `'attention_mask'`, содержащий индикаторные переменные, указывающие на специальные токены. Поскольку длина первого текста оказалась меньше длины второго, то он был дополнен специальным токеном `'PAD'`, которому соответствует индекс 0.

In [ ]:
input_ids

In [ ]:
attention_mask

С учетом специальных токенов первый текст выглядел бы следующим образом:

In [ ]:
tokenizer.decode(encoding['input_ids'][0])

# Обучение BERT

Для обучения модели классификации воспользуемся функцией `BertForSequenceClassification` и оптимизатором `Adam` (из `transformers`). Для дообучения модели BERT шаг рекомендуют выбирать небольшим.

In [ ]:
from transformers import BertForSequenceClassification, AdamW

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', return_dict=True)
model.train()

optimizer = AdamW(model.parameters(), lr=1e-5)

Процесс обучения модели схож с привычным обучением моделей в PyTorch. `BertForSequenceClassification` возвращает два объекта: значение функции потерь и тензор с предсказаниями для двух классов.

In [ ]:
model.zero_grad()
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

loss = outputs.loss
loss.backward()
optimizer.step()

In [ ]:
outputs

Дообучение всех слоев BERT требует значительных вычислительных ресурсов, поэтому стоит зафиксировать параметры энкодера и доообучать только последний слой (принцип transfer learning).

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False

# BERT на реальных данных

Рассмотрим задачу анализа тональности текстов. Скачаем отзывы к фильмам и оценки тональности к ним (1 - позитивный отзыв, 0 - негативный).

In [ ]:
np.random.seed(101)

df = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})
df = df.sample(2000)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df['sentiment'].value_counts().plot(kind='bar')
plt.show()

### Задание

Откройте новый ноутбук на kaggle (слева вкладка Code -> New Notebook). Через "+ Add data" справа найдите и добавьте набор данных IMDB Dataset of 50K Movie Reviews.

Напишите класс `IMDbDataset`, который на вход принимает тексты и метки класса (`texts`, `labels`). Указания:
1. Под `__init__` преобразуйте `pandas.Series` с текстами в список текстов (`tokenizer` требует, чтобы тексты были представлены в виде списка).
2. К полученному списку текстов примените `tokenizer` аналогично примеру выше. Получите `self.input_ids`, `self.attention_masks`, `self.labels = labels`.
3. Метод `__len__` должен возвращать длину датасета (`len(self.labels)`).
4. Метод `__getitem__` должен взвращать `input_ids`, `attention_masks`, и `labels` для каждого примера.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class IMDbDataset(Dataset):
    def __init__(self, texts, labels):
        # Ваш код здесь
        pass

    def __len__(self):
        # Ваш код здесь
        pass
        
    def __getitem__(self, idx):
        # Ваш код здесь
        pass

Следующий код должен выполняться без ошибок, если всё сделано правильно:

In [ ]:
train_dataset = IMDbDataset(texts, df['sentiment'].values)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

5. Определите модель аналогично примеру выше и перенесите её на `device`. Позже не забудьте перед обучением включить GPU в ноутбуке.
6. Обучать будем методом `AdamW`. Установите шаг обучения равным `2e-3`.
7. Количество эпох задайте равным 10.
8. Обучите модель. Каждые 10 мини-батчей выводите среднее значение функции потерь по данным 10 мини-батчам. В отдельной переменной сохраняйте значение функции потерь на каждой из эпох.

In [ ]:
torch.manual_seed(101)

# model = 
model.train()

for param in model.base_model.parameters():
    param.requires_grad = False

# optimizer = 
# n_epochs = 
train_len = len(train_loader)

batch_losses =list()
epoch_losses = list()
for epoch in range(1, n_epochs+1):
    batch_loss = 0
    epoch_loss = 0
    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        model.zero_grad()
        
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.unsqueeze(0)
        labels = labels.to(device)
        
        # Ваш код здесь

Выведите графики процесса обучения `batch_losses` и `epoch_losses`.

In [ ]:
# Ваш код здесь